In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# load the data in a numpy array
D = pd.read_csv('../input/creditcard.csv')
#print(D.loc[0:5,'Class'])
Z = D.values
X = Z[:, 0:30]
Y = Z[:, 30]
print (X.shape)

In [ ]:
import matplotlib.pyplot as plt
count_classes = pd.value_counts(D['Class'], sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Class Distribution")
LABELS = ["Normal", "Fraud"]
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency");

In [ ]:
# Before going any further operations on data, let's right a function to calculate, true positives, 
# false positives, true negatives, precision, recall and F1 score
def calulate_precision_recall(y_test, y_pred):
    tn = 0;
    tp = 0;
    fp = 0;
    fn = 0;
    for i in range(len(y_pred)):
        if( y_test[i] == 0 and y_pred[i].round() == 0):
            tn = tn+1
        if( y_test[i] == 1 and y_pred[i].round() == 0):
            fn = fn+1
        if( y_test[i] == 1 and y_pred[i].round() == 1):
            tp = tp+1
        if( y_test[i] == 0 and y_pred[i].round() == 1):
            fp = fp+1
    prec=0
    rec=0
    F1=0
    if(tp+fp>0):
        prec= tp/(tp+fp)  
    if(tp+fn>0):
        rec = tp/(tp+fn)
    if(prec+rec>0):
        F1 = 2*prec*rec/(prec+rec)
    print('True Positives: ' + str(tp))
    print('True Negatives: ' + str(tn))
    print('False Positives: ' + str(fp))
    print('False Negatives: ' + str(fn))
    print('Precision: ' + str(prec))
    print('Recall: ' + str(rec))
    print('F1 score: ' + str(F1))

In [ ]:
# Split the data into training and test set
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
import tensorflow as tf
from tensorflow import keras

Let's add a neural network and test the efficiency.

In [ ]:
from keras import regularizers
model = keras.Sequential([
    keras.layers.BatchNormalization(axis=-1, input_shape=(30,), momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                                    beta_regularizer=None, gamma_regularizer=None, 
                                    beta_constraint=None, gamma_constraint=None),
    #keras.layers.Dense(50, input_shape=(30,), activation=tf.nn.relu, kernel_initializer='glorot_normal',
     #           kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
   #             kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
     #           kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
               kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    keras.layers.Dense(1, input_shape=(30,), activation=tf.nn.sigmoid, kernel_initializer='random_uniform',
                kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros')
])
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=4)

In [ ]:
# evaluate predictions
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test[:], y_pred.round())
print("Accuracy: %.2f%%" % (accuracy * 100.0))
calulate_precision_recall(y_test, y_pred)

We have achieved a 99.83% accuracy, but it's predicting all as negatives, which is definitely a pickle.

Let's try another approach, try to add a custom loss function. Here we are going to penalise loss function if it predicts a false negative, that is predicting a positive class as negative.

In [ ]:
#Sample custom function implement to add penalise false positives
import keras.backend as K
def custom_skewed_loss(y_true, y_pred):
    loss = (-1) * K.sum((y_true * K.log(y_pred+K.epsilon())) - 100*(y_true*(1-y_pred)) + ((1 - y_true)* K.log(1 - y_pred+K.epsilon())))
    #loss = (1 - y_true)* K.log(1 - y_pred+K.epsilon())
    return loss

In [ ]:
from keras import regularizers
model = keras.Sequential([
    keras.layers.BatchNormalization(axis=-1, input_shape=(30,), momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                                    beta_regularizer=None, gamma_regularizer=None, 
                                    beta_constraint=None, gamma_constraint=None),
    #keras.layers.Dense(50, input_shape=(30,), activation=tf.nn.relu, kernel_initializer='glorot_normal',
     #           kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
   #             kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
     #           kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
               kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros'),
    keras.layers.Dense(1, input_shape=(30,), activation=tf.nn.sigmoid, kernel_initializer='random_uniform',
                kernel_regularizer=regularizers.l2(0.01), bias_initializer='zeros')
])
model.compile(optimizer='adam', 
              #loss='binary_crossentropy',
              loss=custom_skewed_loss,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=4)

In [ ]:
# evaluate predictions
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test[:], y_pred.round())
print("Accuracy: %.2f%%" % (accuracy * 100.0))
calulate_precision_recall(y_test, y_pred)

We are getting a lower accuracy 99.80%, but overall our algorithm is performing much better. Here we can further improve recall(predecting positives) by penalising the function further, if I use coefficient of middle term as 500, I will get below result:
Accuracy: 99.03%
True Positives: 103
True Negatives: 70409
False Positives: 673
False Negatives: 17
Precision: 0.1327319587628866
Recall: 0.8583333333333333
F1 score: 0.2299107142857143
We have become better at predicting true positives, but we are predicting a lot of false positives as well. Here a general rule of thumb can be to try and optimise the loss function to achieve the optimum F1 score. Although we can avoid this rule if missing a positive class is too costly.

Now instead of neural netwrok let's try another algorithm, XGBoost based on gradient boosting.

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:

# make predictions for test data
y_pred = xgb_model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
calulate_precision_recall(y_test, y_pred)

Above is a pretty good result, with higher F1 score, but recall is lower as compared to neural network with custom loss function. We can use either of the two approaches depnding on our requirements.

In [ ]:
import scipy
from scipy.stats import multivariate_normal

In [ ]:
plt.hist((X[:,25]), bins=100)
#2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
#20,21,22,23,24,25,26,27,28

In [ ]:
Z = X[:,2:26]

In [ ]:
def getMeanAndVariance(X):
    (m,n) = X.shape
    # initializing mu(mean) and sigma2(variance)
    mu = np.zeros((n,1))
    sigma2 = np.zeros((n,1))
    # mean
    mu = np.sum(X, axis=0)/m
    tempSigma = X - mu
    tempSigma2 = np.power(tempSigma, 2)
    sigma2 = np.sum(tempSigma2, axis=0)/m
    return (mu, sigma2)
def getCovariance(X):
    return np.cov(X, rowvar=False)
def calculateProbabilities(X, mean, cov):
    # var = multivariate_normal(mean, cov, allow_singular=True)
    var = multivariate_normal(mean, cov)
    p = np.zeros((len(X),1))
    for i in range(len(X)):
        p[i] = var.pdf(X[i])
    return p

In [ ]:
Z_train, Z_test, y_train1, y_test1 = train_test_split(Z, Y)

In [ ]:
(mean, var1) = getMeanAndVariance(Z_train)
cov = getCovariance(Z_train)
p = calculateProbabilities(Z_train, mean, cov)

In [ ]:
p_val = calculateProbabilities(Z_test, mean, cov)
print(p_val.shape)

In [ ]:
predictions = (p_val>0.5).astype(int)
print(predictions.shape)
print(y_train1.shape)
print(len(p_val))

Find the threshold on the basis of F1 scores.

In [ ]:
def select_threshhold(y, p_val):
    p_sort = np.sort(p_val, axis=0)
    length = len(p_val) 
    i = 0
    bestEpsilon = 0
    bestF1 = 0
    F1 = 0
    while i < length: 
        tn = 0;
        tp = 0;
        fp = 0;
        fn = 0;
        predictions = (p_val<p_sort[i]).astype(int)
        for j in range(len(p_val)):
            if( y[j] == 0 and predictions[j,0] == 0):
                tn = tn+1
            if( y[j] == 1 and predictions[j,0] == 0):
                fn = fn+1
            if( y[j] == 1 and predictions[j,0] == 1):
                tp = tp+1
            if( y[j] == 0 and predictions[j,0] == 1):
                fp = fp+1
        i += 1000
        prec=0
        rec=0
        #print(tn)
        #print(tp)
        #print(fp)
        #print(fn)
        #print(i)
        if(tp+fp>0):
            prec= tp/(tp+fp)  
        if(tp+fn>0):
            rec = tp/(tp+fn)
        if(prec+rec>0):
            F1 = 2*prec*rec/(prec+rec)
            if F1 > bestF1:
                bestF1 = F1
                bestEpsilon = p_sort[i];
    tn = 0
    fn = 0
    tp = 0
    fp = 0
    for j in range(len(p_val)):
        if( y[j] == 0 and p_val[j,0] > bestEpsilon):
            tn = tn+1
        if( y[j] == 1 and p_val[j,0] > bestEpsilon):
            fn = fn+1
        if( y[j] == 1 and p_val[j,0] <= bestEpsilon):
            tp = tp+1
        if( y[j] == 0 and p_val[j,0] <= bestEpsilon):
            fp = fp+1
    print('True Positives: ' + str(tp))
    print('True Negatives: ' + str(tn))
    print('False Positives: ' + str(fp))
    print('False Negatives: ' + str(fn))
    print('Precision: ' + str(tp/(tp+fp)))
    print('Recall: ' + str(tp/(tp+fn)))
    print('F1 score: ' + str(2*prec*rec/(prec+rec)))    
    return (bestF1, bestEpsilon)

In [ ]:
(F1, bestEpsilon) = select_threshhold(y_test1, p_val)
print(F1)
print(bestEpsilon)

As we have seen earlier simple neural network classifier, doesn't work with skewd classes. Below is another approach to solve the problem of skewed classes. It's a variant of oversampling known as SMOTE.

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smt = SMOTE()
X_train_s, y_train_s = smt.fit_sample(X_train, y_train)

In [ ]:
print(X_train.shape)
print(X_train_s.shape)
print(y_train.shape)
print(y_train_s.shape)

In [ ]:
from keras import regularizers
model = keras.Sequential([
    keras.layers.BatchNormalization(axis=-1, input_shape=(30,), momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                                    beta_regularizer=None, gamma_regularizer=None, 
                                    beta_constraint=None, gamma_constraint=None),
    keras.layers.Dense(50, activation=tf.nn.relu ,
                       kernel_initializer='random_uniform',bias_initializer='zeros'
                      ),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
     #            bias_initializer='zeros'),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
     #            bias_initializer='zeros'),
    #keras.layers.Dense(50, activation=tf.nn.relu, kernel_initializer='random_uniform',
    #             bias_initializer='zeros'),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.compile(optimizer='sgd', 
              loss='binary_crossentropy',
              #loss=custom_skewed_loss,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train_s, y_train_s, epochs=8)

In [ ]:
# evaluate predictions
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test[:], y_pred.round())
print("Accuracy: %.2f%%" % (accuracy * 100.0))
calulate_precision_recall(y_test, y_pred)